### Graphs!  

actually similar to trees

<b> 127. Word Ladder </b>

only change from word to words that have exactly one letter difference. in this way the problem can be transformed into a graph problem, begin word => visit its neighbors (defined as word with only one char difference) => non-visited neighbors' neighbors => until we arrived at the end word.

so it is a breath first search structure. (BFS)

In [3]:
from collections import deque
class Solution:
    def ladderLength(self, beginWord, endWord, wordList):
        def construct_neighbors(word_list):
            d = {}
            for word in word_list:
                for i in range(len(word)):
                    # no need to worry i+1 is out of bound, cz slicing wont give index out of bound error
                    key = word[:i]+'_'+word[i+1:]
                    d[key] = d.get(key, [])+[word]  # {'_ot': ['lot', 'dot', 'hot']}
            return d
        
        def bfs_words(begin, end, word_dict):
            # queue has word and level => step
            # why initiate the step as 1? => example "hit" -> "hot" -> "dot" -> "dog" -> "cog", result should be 5, not 4
            queue, visited = deque([(begin, 1)]), set()
            while queue:
                word, step = queue.popleft()  # this is why using deque
                if word not in visited:  # this check makes the execution much much faster
                    visited.add(word)
                    if word == end:
                        return step
                    # otherwise find its neighbors and add to queue
                    for i in range(len(word)):
                        s = word[:i] + '_' + word[i+1:]
                        neighbors = word_dict.get(s, [])
                        for n_word in neighbors:
                            if n_word not in visited:
                                queue.append((n_word, step+1))
            return 0
        
        if endWord not in wordList:
            return 0
        word_dict = construct_neighbors(set(wordList))
        return bfs_words(beginWord, endWord, word_dict)
            
            
            
            
            

In [4]:
beginWord = "hit"
endWord = "cog"
word_dict = ["hot","dot","dog","lot","log","cog"]
a= Solution()
a.ladderLength(beginWord, endWord, word_dict)

5

bidirectional BFS. Search both from start word and end word, if they meet at some intermediate word, that means existing such a road from start word to end word

In [9]:
from collections import deque, defaultdict
class Solution:
    def ladderLength(self, beginWord, endWord, wordList):
        """bidirectional BFS"""
        
        def find_words(queue, one_visited, two_visited):
            """
            queue: [(word, level)], deque
            one_visited: {word: level} current direction
            two_visited; {word: level} the other direction
            """
            word, step = queue.popleft()
            for i in range(L):
                intermediate = word[:i]+'_'+word[i+1:]
                
                for neighbor in neighbors_dict[intermediate]:
                    if neighbor in two_visited:
                        return step + two_visited[neighbor]
                    if neighbor not in one_visited:
                        one_visited[neighbor] = step + 1
                        queue.append((neighbor, step+1))
            return
        
        if endWord not in wordList:
            return 0
        
        L = len(beginWord)
        
        # construct neighbors dictionary
        neighbors_dict = defaultdict(list)
        for word in wordList:
            for i in range(L):  # only need to loop L time
                key = word[:i]+'_'+word[i+1:]
                neighbors_dict[key].append(word)
        
        queue_begin = deque([(beginWord, 1)])
        queue_end = deque([(endWord, 1)])
        visited_begin = {beginWord: 1}
        visited_end = {endWord: 1}
        # if one queue is empty the other is not, but they still dont meet, that means no such a way
        while queue_begin and queue_end:
            # queue_begin, queue_end, visited_begin, visited_end are all reference and are updated in side of the
            # execution of find_words() method
            # go forward one level from begin word
            res = find_words(queue_begin, visited_begin, visited_end)
            if res:
                return res
            # go forward one level from end word
            res = find_words(queue_end, visited_end, visited_begin)
            if res:
                return res
        return 0

In [10]:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
b= Solution()
b.ladderLength(beginWord, endWord, wordList)

5

<b> 126 Word Ladder II </b>

still can create a graph,   
one layer over one layer, each layer is a dictionary, key as valid word, value as all possible path whose last value is this valid word  

In [5]:
from collections import defaultdict
def findLadders(beginWord, endWord, wordList):
    def construct_graph(wordList, beginWord):
        l = len(beginWord)
        d = {}
        for word in wordList:
            for i in range(l):
                key = word[:i] + '_' + word[i+1:]
                d[key] = d.get(key, [])+[word]
        return d
    
    if endWord not in wordList:
        return []
    
    result = []
    word_graph = construct_graph(wordList, beginWord)
    wordList = set(wordList)  # set searching is O(1)
    layer = {} # as the queue
    # key: last word in the path, value: list of list of words, each list of words's last word is the key
    layer[beginWord] = [[beginWord]] # a list of possible path end up in beginWord
    while layer:
        next_layer = defaultdict(list)
        for word in layer:
            if word == endWord:
#                 result.extend(path for path in layer[word])
                result.extend(layer[word])
            else:
                # searching neighbors
                for i in range(len(word)):
                    key = word[:i] + '_' + word[i+1:]
                    neighbors = word_graph.get(key, [])
                    for w in neighbors:
                        if w in wordList:
                            next_layer[w]+=[path+[w] for path in layer[word]]
        wordList -= set(next_layer.keys())  # remove visited node => replace it with visited set is okay too
        layer=next_layer
    return result

In [6]:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
findLadders(beginWord, endWord, wordList)

[['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]

<b> 207 Course Schedule </b>

the problem is also known as topological sort problem, which is to find a global order for all nodes in a DAG (Directed Acyclic Graph) with regarding to their dependencies.

<b>Topological sort </b>  
construct a graph:{prerequisite: [dependings]}  
construct a degree dictionary: {course: degree integer}: n degree means this course has n prerequisits need to take  
the course with degree 0 is the one able to take first. if no course has 0 degree, the courses are in a loop, cant be finished.
first starting with courses with degree 0, find its depending courses and reduce their degrees by 1. that means number of requisites needs to be finished before able to take the depending course is reduced by 1. If now a new course has degree 0, add it to the start course queue.  
at the end if all courses are taken, each of the degree is 0, sum is 0 too

In [11]:
from collections import defaultdict, deque
def canFinish(numCourses, prerequisites):
    # construct a graph:{prerequisite: [dependings]}
    graph = defaultdict(list)
    # construct a degree dictionary: {course: degree integer}
    # n degree means this course has n prerequisits need to take
    degree = [0]*numCourses
    for d, pre in prerequisites:
        graph[pre].append(d)
        degree[d]+=1 # course number happens to be used as index

    # the course with degree 0 is the able to take first. if no course has 0 degree, the courses are in a loop, cant be finished
    start_courses = deque([i for i in range(numCourses) if degree[i]==0])
    while start_courses:
        pre = start_courses.popleft()
        for j in graph[pre]:
            # j is the depending courses of i
            degree[j]-=1  # number of requisites needs to be finished before able to take the course j is reduced by 1
            if degree[j]==0:
                # able to take it as next course
                start_courses.append(j)
    # if all courses are taken, they will all be 0
    return not sum(degree)

In [12]:
numCourses = 2
prerequisites = [[1,0],[0,1]]
canFinish(numCourses, prerequisites)

False

<b> 210 course schedule II </b>.   
similar as above

In [14]:
from collections import defaultdict, deque
def findOrder(numCourses, prerequisites): 
    graph = defaultdict(list)  # {prerequisite: [depending courses]}
    degree=[0]*numCourses  # [integer] how many prerequisites needs to take

    for course, pre in prerequisites:
        graph[pre].append(course)
        degree[course]+=1 

    result=[]
    # start with course with no prerequisite
    starting_course = deque([i for i in range(numCourses) if degree[i]==0])
    while starting_course:
        course = starting_course.popleft()
        result.append(course)
        # its dependencies' degree - 1
        for d in graph[course]:
            degree[d]-=1
            if degree[d]==0:  # this also helps avoid class that is already taken
                # able to be taken next
                starting_course.append(d)

    if sum(degree)==0:
        # all courses are taken
        return result
    else:
        return []

In [15]:
numCourses = 4
prerequisites=[[1,0],[2,0],[3,1],[3,2]]
findOrder(numCourses, prerequisites)

[0, 1, 2, 3]

<b> 399 evaluate division </b>

YEAH! I DID IT!  
it is a directed graph problem, using breath first search to search target value layer by layer (neighbor by neighbor), remember to skip the visited node to avoid endless loop

In [10]:
from collections import defaultdict
def calcEquation(equations, values, queries):
    result = [-1.0]*len(queries)
    # construct the graph
    graph = defaultdict(dict)
    # looks like {a: {b, 2}, b:{a: 1/2, c: 3}, c:{b:1/3}}
    for ind, (i, j) in enumerate(equations):
        graph[i][j]= values[ind]
        graph[j][i]= 1/values[ind]

    def helper(x, y):
        # BFS helper function,find path from x to y, given a graph
        queue = [(x,1)] # (node, val)
        visited = set()
        while queue:
            node, val = queue.pop()
            if node in visited:
                continue
            visited.add(node)
            neighbors = graph[node] # a dict {node:value}
            if y in neighbors:
                return val*neighbors[y]
            else:
                for n in neighbors:
                    queue.append((n, val*neighbors[n]))
            # queue = next_queue
        return -1.0

    for ind, (x, y) in enumerate(queries):
        if x not in graph or y not in graph:
            continue
        elif x==y and x in graph[x]:
            result[ind] = 1.0
        elif y in graph[x]:
            result[ind] = graph[x][y]
        else:
            # find the transition path
            # there maybe multiple paths, but since the assumption is there is no contradiction, it is safe to pick random one
            # depth first search, search neighbors and neighbors' neighbors until get the target

            result[ind] = helper(x, y)
    return result

In [11]:
equations = [ ["a", "b"], ["b", "c"] ]
values = [2.0, 3.0]
queries = [ ["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"]] 
calcEquation(equations, values, queries)

[6.0, 0.5, -1.0, 1.0, -1.0]

<b> 947 most stones removed with same row or column </b>

the question is actually finding the number of islands, which are defined as nodes share same row or column. that is if two nodes are common in row or column value, they are in the same island.  
two ways are used here.   
=> Union Find, a new structure  
=> DFS


<b>way 1</b> union find. not too sure how it works still

In [12]:
def removeStones(stones):
    """union find"""
    """
    Connected stones can be reduced to 1 stone,
    the maximum stones can be removed = stones number - islands number.
    Two stones are connected if they are in the same row or same col.
    """
    UF = {}
    def find(x):
        if x!=UF[x]:
            UF[x] = find(UF[x])
        return UF[x]

    def union(x, y):
        UF.setdefault(x,x)
        UF.setdefault(y,y)
        UF[find(x)] = find(y)


    for i, j in stones:
        # need to separate j from i to distinct range, so they can be key
        union(i, ~j)
        # here ~j will make the range for j to [-1, -10001]
    # for element in UF, find its root, the number of unique root is the number of set
    return len(stones) - len({find(x) for x in UF})

<b> way 2</b> DFS way, easier to understand

first constructing the "graph" structure, which is two dict of list, representing rows (rows: [cols share the same rows]) and cols(col: [rows share the same col]).   
then a dfs helper function to discard the point out, and loop through each of the neighbors(points share same row or col). keep track of the island number.  
because each island can only left 1 stone, so the max number of stone can be removed is total stone number - island number

In [13]:
import collections
def removeStones(stones):
    """finding the number of island, then stone number - island number is the largest stone you can remove, so that means each island finally got 1 stone"""

    def dfs_helper(i, j):
        """discard points with same row or col, recursive to loop all points in the same island"""
        points.discard((i, j))  # discard() method wont raise error if node not exist
        # go along the neighbor
        for y in rows[i]:
            if (i, y) in points:
                dfs_helper(i, y)
        for x in cols[j]:
            if (x, j) in points:
                dfs_helper(x, j)

    # points is a set! and will be modified in the dfs helper
    points = {(i, j) for i, j in stones}
    island = 0
    rows = collections.defaultdict(list)
    cols = collections.defaultdict(list)
    # constructing the "graph" structure
    for i, j in stones:
        # {i: [j1, j2], j1:[i]} => like a graph
        rows[i].append(j)
        cols[j].append(i)
    for i, j in stones:
        if (i, j) in points:
            dfs_helper(i, j)
            island += 1
    return len(stones) - island

In [14]:
stones= [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
removeStones(stones)

5

<b> 684 redundant connection </b>  
using union-find structure to detect which one will cause a cycle

In [15]:
def findRedundantConnection(edges):
    """union find problem, to detect if adding something will create a loop"""
    # recall in the tutorial video, the way of using an array to save what is the root parent of each node
    # initiate a list with index corresponding to 1 to N, the position 0 is an extra useless one
    # at beginning, each node is indepent, parent is iteself
    parent = [x for x in range(len(edges)+1)]
    def find(x):
        """path compression"""
        # to find the final root parent recursively
        if parent[x]!=x:
            parent[x] = find(parent[x])
        return parent[x]
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        if root_x == root_y:
            # find two nodes x and y actually will create a loop
            return True
        # assinging root of y to be the final root of root_x, the union action to add this pair to the set
        # constructing the node to the set
        parent[root_x] = root_y 

    result = []  # to make sure the last valid answer is returned
    for x, y in edges:
        if union(x, y):
            result = [x,y]
    return result


In [16]:
edges = [[1,2], [1,3], [2,3]]
findRedundantConnection(edges)

[2, 3]

adding union-by-rank optimization, to choose the node with more elements as the root, it suppose to speed up the finding process

In [ ]:
def findRedundantConnection(edges):
    """union find problem, to detect if adding something will create a loop"""
    # recall in the tutorial video, the way of using an array to save what is the root parent of each node
    # to create a list with index corresponding to 1 to N, the position 0 is an extra useless one
    parent = [x for x in range(len(edges)+1)]
    rank = [0]*(len(edges)+1)
    def find(x):
        # to find the final root parent recursively
        if parent[x]!=x:
            parent[x] = find(parent[x])
        return parent[x]
    def union(x, y):
        root_x = find(x)
        root_y = find(y)
        if root_x == root_y:
            # find two nodes x and y actually will create a loop
            return True
        # assinging root of y to be the final root of root_x, the union action to add this pair to the set
        elif rank[root_x] > rank[root_y]:
            # root_x has more children elements than root_y
            parent[root_y] = root_x
        elif rank[root_y] > rank[root_x]:
            parent[root_x] = root_y 
        else:
            parent[root_y] = root_x
            rank[root_x]+=1

    result = []
    for x, y in edges:
        if union(x, y):
            result = [x,y]
    return result


<b> 1462 Course Schedule IV </b>

a similar way to other course schedule problem.  
It is much quicker than the following way.

Time: O(P * N), N for topological sort and P for passing all prerequisites.
Space: O(N^2) for all sets.

In [3]:
import collections
def checkIfPrerequisite(n, prerequisites, queries):
    graph = collections.defaultdict(list) # save {pre: [course]}
    degree = [0]*n # for how many prequisites needed for course at index i
    pres = [set() for _ in range(n)] # save what prerequistes are for each of the course denoted as index i. for example, course i (the index) has a set of prerequisites.

    for pre, course in prerequisites:
        graph[pre].append(course)
        degree[course]+=1
        pres[course].add(pre)

    queue = collections.deque(course for course, d in enumerate(degree) if d==0)
    while queue:
        pre = queue.popleft()
        for course in graph[pre]:
            degree[course]-=1
            # union operation, get distincts prerequisites set for course
            # course's preresites's prefrequisists are the course's prerequisites too
            pres[course] = pres[course] | pres[pre]
            if degree[course] == 0:
                # no more prerequisites needed, able to be taken next
                queue.append(course)
    # the check operation in set is O(1)
    return [pre in pres[course] for pre, course in queries]

In [4]:
n = 5
prerequisites = [[0,1],[1,2],[2,3],[3,4]]
queries = [[0,4],[4,0],[1,3],[3,0]]
checkIfPrerequisite(n, prerequisites, queries)

[True, False, True, False]

another way  

a graph problem, to check if two nodes are connected in a particular direction. It is a directed graph. Floyd-Warshall O(n^3) is an algorithm that will output the minium distance of any vertices.
construct a n*n matrix to store the relationship between each node, [i][j] means from i to j, with direction information, is different from [j][i]
Time complexity O(N^3)

In [1]:
def checkIfPrerequisite(n, prerequisites, queries):
    '''a graph problem, to check if two nodes are connected in a particular direction
    construct a n*n matrix to store the relationship between each node, [i][j] means from i to j, with direction information, is different from [j][i]
    O(N^3)
    '''
    matrix = [[False]*n for _ in range(n)]
    # add the obvious prerequisites relationship to the matrix 
    for i, j in prerequisites:
        matrix[i][j] = True
    # add indirect prerequisites relationship to the matrix
    for k in range(n):
        for i in range(n):
            for j in range(n):
                matrix[i][j] = matrix[i][j] or (matrix[i][k] and matrix[k][j])

    res = [matrix[i][j] for i, j in queries]
    return res

In [2]:
n = 5
prerequisites = [[0,1],[1,2],[2,3],[3,4]]
queries = [[0,4],[4,0],[1,3],[3,0]]
checkIfPrerequisite(n, prerequisites, queries)

[True, False, True, False]

<b> 332 reconstruct itineray </b>

DFS way, together using sort to ensure the lexical order.  
the graph object: {'start airport' : [a list of airport in reverse lexical order]} when pop, airport with smaller lexical order will be visited first

In [3]:
from collections import defaultdict
def findItinerary(tickets):
    """DFS"""
    graph = defaultdict(list)
    for a, b in sorted(tickets, reverse=True):
        # sort tickets so {a:[b]} [b] is in reversed lexical order, when pop, airport with smaller lexical order will be visited first
        graph[a].append(b)
    route = []
    def visit(airport):
        """airport: str
        the DFS helper
        """
        while graph[airport]:
            visit(graph[airport].pop())
        route.append(airport)
        # the route append the airport from the bottom of the graph, so at last need to reverse it back

    visit('JFK')
    return route[::-1]

In [4]:
tickets = [["JFK","SFO"],["JFK","ATL"],["SFO","ATL"],["ATL","JFK"],["ATL","SFO"]]
findItinerary(tickets)

['JFK', 'ATL', 'JFK', 'SFO', 'ATL', 'SFO']